# plan for public bank account
- download transactions from bank in column-separated value format
- import these into a double entry plain text accounting software such as `beancount` 
- remove personally identifiable information from the transactions
- publish the transactions in a public repository
- translate the transactions from the public repository to an HTML page, onefact.org/accounting (with code at https://github.com/onefact/onefact.org/blob/main/pages/accounting.md)

# references
- example importer for utrade: https://github.com/beancount/beancount/blob/v2/examples/ingest/office/importers/utrade/utrade_csv.py
- public bank accounts: https://ib.fio.cz/ib/transparent?a=2002470101 
- all the beauty and the bloodshed (sacklers donating to the met; https://www.newyorker.com/magazine/2017/10/30/the-family-that-built-an-empire-of-pain)
- FTX and effective altruism (https://www.youtube.com/watch?v=Lm0vHQYKI-Y&t=1s)

In [4]:
import csv
from datetime import datetime
from decimal import Decimal as D
from money import Money

def extract(self, file):
    # Open the CSV file and create directives.
    entries = []
    with open(file, newline='') as infile:
        reader = csv.DictReader(infile)
        for index, row in enumerate(reader):
            meta = data.new_metadata(file, index)
            date = datetime.strptime(row['Date (UTC)'], '%m/%d/%Y').date()
            desc = row['Description']
            amount = Money(D(row['Amount']), 'USD')
            units = amount if amount > 0 else Money(0, 'USD')
            fees = amount if amount < 0 else Money(0, 'USD')
            other = units - fees
            entry = data.Entry(meta, date, desc, other)
            entry.add_trade(self.account, other, fees, self.cost)
            entries.append(entry)
    return entries


ModuleNotFoundError: No module named 'money'

Prompt for bing:

> Hi Bing! Can you please rewrite this python code to use the csv file called "quickbooks-export-onefact-2023-02-01--2023-02-28.csv" and parse the correct types of transactions? The file is in QuickBooks format and the headers are "Date (UTC),Description,Amount,Status,Source Account,Bank Description,Reference,Note". The following imports must be used: ``` from beancount.core.number import D from beancount.core.number import ZERO from beancount.core import data from beancount.core import account from beancount.core import amount from beancount.core import position from beancount.ingest import importer ``` The python code: ``` def extract(self, file): # Open the CSV file and create directives. entries = [] index = 0 with open(file.name) as infile: for index, row in enumerate(csv.DictReader(infile)): meta = data.new_metadata(file.name, index) date = parse(row['DATE']).date() rtype = row['TYPE'] link = "ut{0[REF #]}".format(row) desc = "({0[TYPE]}) {0[DESCRIPTION]}".format(row) units = amount.Amount(D(row['AMOUNT']), self.currency) fees = amount.Amount(D(row['FEES']), self.currency) other = amount.add(units, fees) ```


In [11]:
from beancount.core.number import D
from beancount.core.number import ZERO
from beancount.core import data
from beancount.core import account
from beancount.core import amount
from beancount.core import position
from beancount.ingest import importer

def extract(file):
        # Open the CSV file and create directives.
        entries = []
        index = 0
        with open(file.name) as infile:
            # Use csv.DictReader to read the file with headers
            reader = csv.DictReader(infile)
            # Loop through each row in the file
            for index, row in enumerate(reader):
                meta = data.new_metadata(file.name, index)
                # Parse the date from the first column
                date = parse(row['Date (UTC)']).date()
                # Get the description from the second column
                desc = row['Description']
                # Get the amount from the third column and convert it to a decimal
                units = amount.Amount(D(row['Amount']), self.currency)
                # Get the status from the fourth column
                status = row['Status']
                # Get the source account from the fifth column
                source = row['Source Account']
                # Get the bank description from the sixth column
                bank_desc = row['Bank Description']
                # Get the reference from the seventh column
                ref = row['Reference']
                # Get the note from the eighth column
                note = row['Note']
                # Create a link from the reference
                link = "ut{}".format(ref)
                # Depending on the type of transaction, create different directives
                if status == "Deposit":
                    # Create an income directive
                    entry = data.Transaction(
                        meta,
                        date,
                        "*",
                        desc,
                        bank_desc,
                        data.EMPTY_SET,
                        {link},
                        [
                            data.Posting(source, units, None, None, None, None),
                            data.Posting(self.income_account, -units, None, None, None, None),
                        ],
                    )
                elif status == "Withdrawal":
                    # Create an expense directive
                    entry = data.Transaction(
                        meta,
                        date,
                        "*",
                        desc,
                        bank_desc,
                        data.EMPTY_SET,
                        {link},
                        [
                            data.Posting(source, units, None, None, None, None),
                            data.Posting(self.expense_account, -units, None, None, None, None),
                        ],
                    )
                elif status == "Transfer":
                    # Create a transfer directive
                    entry = data.Transaction(
                        meta,
                        date,
                        "*",
                        desc,
                        bank_desc,
                        data.EMPTY_SET,
                        {link},
                        [
                            data.Posting(source, units, None, None, None, None),
                            data.Posting(self.transfer_account, -units, None, None, None, None),
                        ],
                    )
                else:
                    # Skip any other type of transaction
                    continue
                # Add the note as a comment to the directive
                if note:
                    entry.comment = note
                # Add the directive to the list of entries
                entries.append(entry)
        # Return the list of entries
        return entries


In [15]:
# extract entries from a CSV file
from pathlib import Path
extract(file=Path("./bank-statements/quickbooks-export-onefact-2023-03-01--2023-03-31.csv"))

FileNotFoundError: [Errno 2] No such file or directory: 'quickbooks-export-onefact-2023-03-01--2023-03-31.csv'